# Week 6 Exercises

## Ex 0: Hand In 1 Walk Through

## Ex: 1 Install and test Pytorch
Later in the course we will be using the deep learning frame work pytorch. So install it. Install torchviz too to help plot computation graphs.
(package name is torch i.e. pip3 install torch),.

In this exercise your task is to automatic differentation to evaluate derivatives of the sigmoid funcion. 
Let us use automatic differentation for another gradient descent algorithm.

Lets do a similar exercise to last time just with some data and Linear Regression Gradient Descent.
First we need to understand that to represent data we must use torch tensors. Tensors are very much like numpy arrays just with some extra functionality.
The thing we will consider is the backward function that computes gradients of whatever computation you have made using torch tensors.

To see how this works, lets see an example.
Lets evaluate the gradient of the sigmoid function without actually knowing the formula. All you need to know is how to compute the function using standard functions i.e. $s(z) = 1/(1+e^{-z})$.



In [ ]:
import torch
z = torch.zeros(1, requires_grad=True)
sz = 1.0 / (1+ torch.exp(-z))
sz.backward() # compute gradient of sz relative to z in this case
print('Gradient of f(z)=1/(1 + e^{-z}) at z = 0', z.grad)


## Ex 1 Continued: Gradient Descent with pytorch
In deep learning frameworks all we usually need to do is to show how to compute the cost in a given point and the system then automatically computes the gradient in that data point for you. We will se how later in this course, in this exercise we will try and see if we can use this functionality to do gradient descent.
The example will be similar to last weeks gradient descent except now we actually make a data set to run gradient descent on for Linear Regression.


**Setup:**
We create a data set $D$ that will consist of $n=100$ data points in 2D $(x_1,x_2)$ i.e. two features $x_1, x_2$. 
The data feature vectors of $x_1$ and $x_2$ are made orthogonal and $x_1$ has unit norm while $x_2$ has norm $a$.

We generate a target vector 
$$
y = x_1+x_2
$$ 
which is also a vector of length $n$ i.e. the data we are trying to fit comes from *(a very simple)* linear model.

Remember linear regression the in sample error 
$$
\textrm{E}_\textrm{in}(w) = \frac{1}{n} \sum_{i=1}^n (w^\intercal x_i - y_i)^2 = \frac{1}{n} \|Xw -y\|^2
$$


We have written the code to generate data and the surrounding Gradient Descent for loop, all you need is to write the code for computing the cost (ein).
You can only use commands from torch here (no numpy), but you can use standard operators like $+,-,*,/,**$ on torch tensors that work like their numpy equivalent and torch.sum may be very handy
**Complete the gradient descent code below by computing cost using standard operations and torch commands only**

The gradient descent will start the search at $w=(42, 2)$ for some reason. We have also sat an almost arbitrary learning rate. You can change both if you like.


To see how this linear regression exercise relates to the gradient descent exercie from last week try increasing the value of $a$. 

**For the $a$ specified, the lecturer gets an epoch 41 Cost of approximately 0.25 and a w of approximately [5.75, 0.25]**

In [ ]:
import numpy as np 
import torch # the torch tensor library

## CREATE SOME DATA
n = 100
x1 = np.random.rand(n)
x2 = np.random.rand(n)
## Grahm schmidt process - ignore
x1 = x1/np.linalg.norm(x1)
x2 = x2/np.linalg.norm(x2)
x2 = x2 - np.dot(x1, x2) * x1 #
x2 = x2/np.linalg.norm(x2)

## CREATE THE DATA MATRIX
a = 4.0
D = np.c_[x1, a*x2]
# CREATE TARGET FUNCTION VECTOR
y = x1 + x2

# MAKE TORCH VARIABLES TO USE
X = torch.from_numpy(D).double()
ty = torch.from_numpy(y).double()
ni = torch.tensor(1./n, dtype=torch.double)

def torch_gd():
    w = torch.tensor([42.0, 2.0], dtype=torch.double)
    lr = torch.tensor(10.0/a, dtype=torch.double)
    epochs = 42
    cost_hist = []
    for i in range(epochs):
        w.requires_grad_() # say i want gradient relative to w in upcoming computation
        cost = None
        ### YOUR CODE HERE - Compute Ein for linear regression as function of w and store in cost 1 - 4 lines
        ### END CODE
        cost_hist.append(cost)
        print('epoch {0}: Cost {1}'.format(i, cost))
        cost.backward() # compute gradient cost as function of w
        w = w - lr * w.grad # update w
        w.detach_() # removes w from current computation graph
    print('best w found', w)
torch_gd()

## Exercise 2: Backpropagation on small example, manually

Consider the simple neural net 
$$
\textrm{nn}(x_1,x_2) = w_3 \cdot \textrm{relu}(w_1 \cdot x_1 + w_2 \cdot x_2)
$$
where $\textrm{relu}(x) = \max(x,0)$ and $w_1, w_2, w_3$ are the weights of the network

We only have one input point $x=(x_1,x_2) = (3.0, 1.0)$ with target value $y=9.0$

Consider the least squares error 
$$
e=(y-\textrm{nn}(x))^2
$$

We need to minimize the error over $w_1, w_2, w_3$ and will do that using the gradients of $e$ relative to $w_1, w_2, w_3$.
Initialize $w_1=1,w_2=2, w_3=1$

Draw the computational graph for $e$ and run the forward pass to compute the output of the neural network on $x$ and the least squares error, and then run the backwards pass to compute the partial derivative of the loss relative to $w_1,w_2, w_3$ on the fixed input $x,y$ with the fixed values for $w_1,w_2,w_3$.

Write the python code that performs the forward and backwards pass below and evaluate the cost and the gradient
using notation similar to
http://cs231n.github.io/optimization-2/

Print intermediate steps in both the forward and the backward pass.

In [ ]:
print('Do the python forward and backward pass here')
x1 = 3.0
x2 = 1.0 
y = 9.
w1 = 1.0
w2 = 2.0
w3 = 1.0
### YOUR CODE HERE
### END CODE



## Ex 3: Automated Backpropagation using pytorch
In this exercise we will check our results from the previous exercise using pytorch.

For this we only need to code the forward pass and let automatic differentation take care of the rest!

**Task:** Write the forward pass in the cell below and use automatic differentation to test your answer from above.

Use x.retain_grad() to keep the gradient of any intermediate computation used in the forward pass to compare with above



In [ ]:
import torch
from torchviz import make_dot # install this package 

x1 = torch.tensor([[3.]])
x2 = torch.tensor([[1.]])
y = torch.tensor([9.])
W1 = torch.tensor([[1.]], requires_grad=True)
W2 = torch.tensor([[2.]], requires_grad=True)
W3 = torch.tensor([[1.]], requires_grad=True)
### YOUR CODE HERE - The clamp function may be usefull
### END CODE
# print the graph - change naming appropriately
print('d_w1', W1.grad)
print('d_w2', W2.grad)
print('d_w3', W3.grad)
print('Lets show the computation graph')
make_dot(loss, params={'W1': W1, 'W2': W2, 'W3': W3})

## Ex. 4: Matrix of Derivatives for applying function elementwise to vector 
In this exercise we consider the matrix of derivatives when mapping a vector of size $n$ to a vector of size $n$ by applying a function $f$ to each entry in the input.

Let $f$ be a smooth function from $\mathbb{R} \rightarrow \mathbb{R}$.

Let $F: \mathbb{R}^d \rightarrow \mathbb{R^d}$ defined as $F(v) = \left[f(v_1), \dots, f(v_d)\right]$

- Show that the Matrix of Derivative $\frac{\partial F}{\partial x}= D_f$, the $d \times d$ diagonal matrix where the $i$'th diagonal entry is $f'(v_i)$.
- If $f(x) = 1/(1+e^{-x})$ i.e. the logistic sigmoid function then how does the Diagonal matrix look
- If $f(x) = \max(0, x)$ i.e. relu how does the Diagonal Matrix look
- Consider a neural net backpropagation step and let hout be the output of applying F to the vector hin and assume we have computed the vector $\frac{\partial L}{\partial \textrm{hout}}$ ($1 \times h$)
    and wish to compute  $\frac{\partial L}{\partial \textrm{hin}}$ (also $1\times h)$. Write the python code that achieves this below.
    

**hint:** There may be a more efficient way than actually creating the diagonal matrix and multiplying on the backpropagated derivative but you do not have to optimize.

In [ ]:
import numpy as np
h_in = np.array([[-1, 2, 4]])
d_hout = np.array([[1,2,3]])
print('shapes:', h_in.shape, d_hout.shape)
def relu_grad(d_hout, hin):
    d_hin = None
    ### YOUR CODE HERE
    ### END CODE
    return d_hin

def sigmoid_grad(d_hout, hin):
    d_hin = None
    ### YOUR CODE HERE
    ### END CODE
    return d_hin

print('d_hin relu:', relu_grad(d_hout, h_in))
# should be [0, 2, 3]
print('d_hin sigmoid:', sigmoid_grad(d_hout, h_in))
# should be ~ [0.196..., 0.209..., 0.052...]


# Ex 5: Neural Nets by automatic differentation in pytorch
Before we introduce our selves to the idiomatic way of writing neural net code in pytorch we will implement a pair of basic basic neural nets and train it using just automatic differentation and the optim module from pytorch that implements different gradient based optimization methods.

**The task is:**

Using pytorch implement Linear Regression with a weight decay.
(named Ridge Regression in litterature ) using Gradient Descent as Learning Algorithm 
and implement a one hidden layer neural net with relu activation for regression (identity output neuron) with with the same cost

Let $w, b$ be the parameters of the function $f$ we are trying to learn, then least squares with weight decay cost for weight decay parameter $c$ is
$$
\frac{1}{n} \sum_{i=1}^n (f(x)-y)^2 + c \sum_{i=1}^d w_i^2
$$
Note we do not penalize the bias weights.

1. Implement Linear Regression with Gradient Descent and test on the Boston data set for house prices (see cell below)
   Linear regression is simply a neural net with no hidden layer and one output neuron. We have implemented the fit methods to show you how to use the optim module.
   That means all you need to do is implement the cost method.
2. Implement  implement a standard one hidden layer neural net for regression. That means identity output activation and least squares error with weight decay. The formula for weight decay is the same, i.e. you need to add the squared weight to the error for all weights in all layers. 
For nonlineariry use relu. See cell two below
   In this exercise you need to complete the following two methods (hint: fit is similar to linear regression fit(
   - cost - compute the least squares cost of the network on data and return the pytorch tensor of that
   - fit - train 100 steps og gradient descent using optim package - find a good learning rate your self

We test your implementation on a standard regression data set for house prices and compare to the sklearn build in Ridge Regression. 
We only consider in sample error, which is of course not what we care about in the real world!.

Your Linear Regression implementation should get close to the sklearn Ridge Regression implementation that we have included

For the Neural Net you should do better (in sample)

In [ ]:
import torch
from torch import optim
from sklearn.datasets import load_boston
from sklearn import linear_model

print('*'*5, 'Load and Prepare Data', '*'*5)
dataset = load_boston()
# print('dataset', dataset)
X, y = dataset.data, dataset.target
X = (X - X.mean(axis=0))/(X.std(axis=0))
#print('data stats', X.shape, X.mean(axis=0), X.std(axis=0))
ridge=linear_model.Ridge(alpha=0.1, fit_intercept=True)
ridge.fit(X, y)
# print(ridge.coef_, ridge.intercept_)
print('\n', '*'*5, 'Test Sklearn Ridge Regression for Comparison', '*'*5)
print('Ridge Regression Score:', ((ridge.predict(X)-y)**2).mean())

print('\n', '*'*5, 'Make data to torch tensors', '*'*5)
tX = torch.from_numpy(X).float()
ty = torch.from_numpy(y).float().view(-1, 1)


class LR():
    
    def __init__(self):
        pass
        
    def cost(self, X, y, w, b, c=0):
        """ Compute Regularized Least Squares Loss
        
          X: torch.tensor shape (n, d) - Data
          y: torch.tensor shape (n, 1) - Targets
          w: torch.tensor shape (d, 1) - weights
          b: torch.tensor shape (1, 1) - bias weight
          c: scalar, weight decay parameter 
          
          returns (regularized) cost tensor        
        """
        loss = None
        ### YOUR CODE HERE
        ### END CODE
        return loss + c * reg_loss
    
    def fit(self, X, y, c=0):
        """ GD Learning Algorithm for Ridge Regression with pytorch
        
        Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         c: ridge regression weight decay parameter (lambda)
        """
        w = torch.zeros(X.shape[1], 1, requires_grad=True)
        b = torch.zeros(1, 1, requires_grad=True)
        sgd = optim.SGD(params={w, b}, lr=0.1)
        for i in range(100):
            sgd.zero_grad()
            loss = self.cost(X, y, w, b, c=c)
            if i % 10 == 0:
                print('epoch:', i, 'least squares (regularized loss)', loss.item())
            loss.backward()
            sgd.step()
        self.w = w.clone()
        self.b = b.clone()


    def score(self, X, y):
        """ Compute least squares cost for model 
        
        Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         
        returns least squares score of model on data X with targets y
        """
        score = self.cost(X, y, self.w, self.b, c=0)
        return score

print('\n', '*'*5, 'Run Torch Linear Regression Gradient Descent', '*'*5)

tlr = LR()
tlr.fit(tX, ty, 0.1)
print('pytorch Linear Regression Regression least squares score:', tlr.score(tX, ty).item())

In [ ]:
import torch
from torch import optim
from sklearn.datasets import load_boston
from sklearn import linear_model

dataset = load_boston()
X, y = dataset.data, dataset.target
X = (X - X.mean(axis=0))/(X.std(axis=0))
tX = torch.from_numpy(X).float()
ty = torch.from_numpy(y).float().view(-1, 1)

class NN():
    
    def __init__(self):
        pass

    def cost(self, X, y, W1, b1, W2, b2, c=0):
        """ Compute (Regularized) Least Squares Loss of neural net
        The clamp function may be usefull
        
          X: torch.tensor shape (n, d) - Data
          y: torch.tensor shape (n, 1) - Targets
          W1: torch.tensor shape (d, h) - weights
          b1: torch.tensor shape (1, h) - bias weight
          W2: torch.tensor shape (h, 1) - weights
          b2: torch.tensor shape (1, 1) - bias weight
          c: ridge regression weight decay parameter 
    
        returns (weight decay) cost tensor
        """
   
        loss = None
        ### YOUR CODE HERE
        ### END CODE
        return loss
    
    def fit(self, X, y, hidden_size=32, c=0.01):   
        """ GD Learning Algorithm for Ridge Regression with pytorch
        
         Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         hidden_size: int 
         c: float weight decay parameter (lambda)
        """
        input_dim = X.shape[1]        
        W1 = torch.randn(input_dim, hidden_size, requires_grad=True)
        b1 = torch.randn(1, hidden_size, requires_grad=True)
        W2 = torch.randn(hidden_size, 1, requires_grad=True)
        b2 = torch.randn(1, 1, requires_grad=True)
        ### YOUR CODE HERE
        ### END CODE
        self.W1 = W1
        self.W2 = W2
        self.b1 = b1
        self.b2 = b2
        
    def score(self, X, y):
        """ Compute least squares cost for model 
        
        Args:
         X: torch.tensor shape (n, d)
         y: torch.tensor shape (n, 1)
         
        returns least squares score of model on data X with targets y
        """
        score = self.cost(X, y, self.W1, self.b1, self.W2, self.b2, c=0)
        return score


net = NN()
net.fit(tX, ty, hidden_size=16, c=0.01)
print('pytorch Neural Net Regression least squares score:', net.score(tX, ty).item())